# Introduction to Docking

## What is docking?

    1. Predict the orientation and conformation of a small molecule (ligand) in the binding site of the target protein 
    
    2. Estimate its binding affinity. 

## Application scenario of different docking methods

<img src='figure/model_comp.png' width=80%></img>

    1. In small-scale (i.e., dozens of) docking experiments, it is always a good idea to use sophisticated docking methods, such as Flexible CDOCKER
    
    2. However, in large-scale docking experiments, such as virtual screening for lead compound identification, direct application of such methods might not be appropriate or realistic because of its relative large computational cost. 
    
        1. This leads to the development of rigid receptor -- rigid ligand docking method (FFT docking).
        
        2. This also motivates us to development hierarchical docking methods. 

<img src="figure/hierarchical_model.png" width="65%"/></img>

# Introduction to CDOCKER

## CDOCKER is a CHARMM based docking algorithm

> ###  1. Grid based docking method
    
> ###  2. Uses physics based scoring function

<img src="figure/cdocker.png" width="80%"/>

## Motivation for pyCHARMM CDOCKER

> ### 1. Docking is a powerful computational tool and has been applied for various structure-based structure- function explorations. However, the scripting languages of these docking methods are relatively complicated for ones with little knowledge of docking. 
    
> ### 2. On the other hand, to perform a successful docking experiment one typically would need various cheminformatics tools either for pre-processing data or for post analysis.

## Typical performance for CHARMM CDOCKER
<div class='alert alert-block alert-info'>
    <b>From:</b>
X Ding, Y Wu, Y Wang, JZ Vilseck and CL Brooks III. Accelerated CDOCKER with GPUs, Parallel Simulated Annealing, and Fast Fourier Transforms. <i>J Chem Theor Comput</i>, 2020, <b>16</b>:3910-3919. 
</div>
<img src="figure/SB2012.png" width="65%"/>

# General Set Up for pyCHARMM CDOCKER

### Step 1. Import essential python libraries and create a test directory to run this part of the tutorial.

In [1]:
import os
if os.getenv('CHARMM_LIB_DIR') == None:
    # set path to CHARMM_LIB_DIR
    pyCHARMM_LIB = '/home/brookscl/charmm/c47-dev-release/install-pycharmm-c1201-52/lib/'
    os.environ['CHARMM_LIB_DIR'] = pyCHARMM_LIB
import pycharmm
import pycharmm.lib as lib
import pycharmm.read as read
import pycharmm.lingo as lingo
import pycharmm.settings as settings
home = os.getcwd()
# test directory for rigid docking
if not os.path.exists('../test_rigid'): os.mkdir('../test_rigid')
os.chdir('../test_rigid')
rigid = os.getcwd()
print(home,rigid)

/home/brookscl/pyCHARMM-Workshop/CDOCKER_WYu/jupyter_lab /home/brookscl/pyCHARMM-Workshop/CDOCKER_WYu/test_rigid


### Step 2. read in topology and parameter files
> 1. In the following example, only protein and CGENFF topology and paramter files are read.
> 2. If you want to dock against non-protein target, you should read in the corresponding topology and parameter files
> 3. Alternatively, you could read in all topology and parameter files. Then you don't need to set bomblev. 
#### In this tutorial example we first move the needed files for protein and ligand and docking grids to the woking directory

In [2]:
# Move ligand.pdb, ligandrtf, protein.pdb and protein.psf to working directory
os.system(f'cp ../rigid/ligand* {rigid}/')
os.system(f'cp ../rigid/protein* {rigid}/')
# Finally, move the pre-compiuted grid files to the working directory
os.system(f'cp {home}/grid/rigid/* {rigid}/')

0

#### Now read in topology and parameter files

In [3]:
settings.set_bomb_level(-1)
owl = settings.set_warn_level(-5)
read.rtf('"../Toppar/top_all36_prot.rtf"')
read.rtf('"../Toppar/top_all36_cgenff.rtf"', append = True)
read.prm('"../Toppar/par_all36m_prot.prm"', flex = True)
read.prm('"../Toppar/par_all36_cgenff.prm"', append = True, flex = True)
settings.set_bomb_level(0)
read.stream('ligandrtf')
settings.set_warn_level(owl)

  
 CHARMM>     read rtf card -
 CHARMM>     name "../Toppar/top_all36_prot.rtf"
 VOPEN> Attempting to open::../Toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name "../Toppar/top_all36_cgenff.rtf" -
 CHARMM>     append
 VOPEN> Attempting to open::../Toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD V. 4.3  

-5

### Step 3. define grid box information

> 1. grid box center (xyz-coordinates): xcen, ycen, zcen
> > - this is general the center of your targeted binding site
> > - values for this system are stored in the files `xcen, ycen, zcen` in the subdirectory `rigid`
> 2. size of grid box: maxlen
> > - this is this size of the grid volume to dock in, generally several A greater than the maximum dimension of the ligand
> > - the value for this system is stored in the file `maxlen` in the subdirectory `rigid`

<div class='alert alert-block alert-info'>
<b>Note: If we weren't using pre-computed grids we would set flag_grid = True in the Rigid_CDOCKER call below.</b>
</div>

### Step 4. general parameters for the pyCHARMM CDOCKER

# pyCHARMM Rigid CDOCKER

## Helper function

In [4]:
from pycharmm.cdocker import Rigid_CDOCKER
help(Rigid_CDOCKER)

Help on function Rigid_CDOCKER in module pycharmm.cdocker:

Rigid_CDOCKER(xcen=0, ycen=0, zcen=0, maxlen=10, dielec=3, rcta=0, rctb=0, hmax=0, flag_grid=False, flag_rdie=True, flag_form=False, flag_delete_grid=True, probeFile='"../Toppar/fftdock_c36prot_cgenff_probes.txt"', softGridFile='grid-emax-0.6-mine--0.4-maxe-0.4.bin', hardGridFile='grid-emax-3-mine--30-maxe-30.bin', nativeGridFile='grid-emax-100-mine--100-maxe-100.bin', receptorPDB='./protein.pdb', receptorPSF='./protein.psf', ligPDB='./ligand.pdb', ligSeg='LIGA', confDir='./conformer/', placementDir='./placement/', exhaustiveness='high', numPlace=100, numCopy=1000, flag_delete_conformer=True, flag_delete_placement=True, flag_save_all=True, flag_save_cluster=True, flag_save_top=True, flag_suppress_print=True, flag_center_ligand=True, flag_fast_grid=False, flag_use_hbond=False, flag_fast_placement=True, threshold=2500, sort_energy='total_energy', saveDir='./dockresult/')
    Rigid CDOCKER standard docking method 
    
    Parame

## pyCHARMM Rigid CDOCKER Docking and Analysis
### Running Rigid CDOCKER with Default Input
> 1. ligand file: <font color='red'>ligand.pdb</font>
> 2. ligand segment ID: <font color='red'>LIGA</font>
> 3. receptor file : <font color='red'>protein.pdb</font> and <font color='red'>protein.psf</font>
> 4. For docking with Rigid CDOCKER, the user needs to provide a pre-computed ligand conformer library. 
> > - This is generally created by operating on <font color='red'>`ligand.pdb`</font> with `obrotamer`, part of `openbabel`. 
> > - Default folder name: <font color='red'>conformer/</font>
> 5. Default docking trials: <font color='red'>100</font> docking trials per conformer

## Generate 10 rotomeric states of `ligand.pdb` with obrotamer

In [5]:
if not os.path.exists('conformer'): os.mkdir('conformer')
for r in range(1,11):
    os.system(f'obrotamer ligand.pdb > conformer/{r}.pdb')

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

 Number of rotatable bonds: 8
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

With default input the user only needs to provide <font color='red'>grid box information</font> (see example below).

```Rigid_CDOCKER(xcen = 1, ycen = 1, zcen = 1, maxlen = 10)```

### pyCHARMM Rigid CDOCKER Example

We have shown some basic requirments for Rigid CDOCKER. Below is a practical example of perform pyCHARMM Rigid CDOCKER docking experiments.
    
<div class='alert alert-block alert-info'>
<b>Note:</b>
    
> 1. In this example, we specified the <font color='red'>file location</font> of the ligand and receptor (i.e., they are not at their default location).
> 2. We also change the number of docking trials per conformer to <font color='red'>20</font> to reduce the computational cost.
    
</div>

## Now carry-out flexible ligand rigid receptor docking with the `Rigid_CDOCKER` module

In [6]:
## Import module
from pycharmm.cdocker import Rigid_CDOCKER

## File name and pathway
ligPDB = "ligand.pdb"
ligandrtf = "ligandrtf"
confDir = "conformer/"
receptorPDB = "protein.pdb"
receptorPSF = "protein.psf"

## Rigid CDOCKER standard docking protocol
clusterResult, dockResult = Rigid_CDOCKER(xcen = 12.33, ycen = 33.48, zcen = 19.70,
                                        maxlen = 25.762, ligPDB = ligPDB, receptorPDB = receptorPDB,
                                        receptorPSF = receptorPSF, confDir = confDir, flag_grid = True,
                                        flag_delete_conformer = False, numPlace = 5)

print(clusterResult)
print(dockResult)

 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
Ligand placement  1
Ligand placement  2
Ligand placement  3
Ligand placement  4
Ligand placement  5
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
Ligand placement  6
Ligand placement  7
Ligand placement  8
Ligand placement  9
Ligand placement  10
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
Ligand placement  11
Ligand placement  12
Ligand placement  13
Ligand placement  14
Ligand placement  15
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
 *** LEVEL  1 WARNING *** BOMLEV IS    0
Ligand placement  16
Ligand placement 

 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.52 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.56 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.55 Angstroms and starts with atom:       1
   Please check group boundary definitions.
Minimize OMMD ligand copy 6
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.31 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.03 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.03 Angstroms and starts with atom:       1
   Please check group boundary definit

 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.30 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.30 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.32 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.38 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.38 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       14.38 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum gro

 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.36 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.36 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.34 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.38 Angstroms and starts with atom:       1
   Please check group boundary definitions.
Minimize OMMD ligand copy 22
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.86 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.99 Angstroms and starts with atom:       1
   Please check group boundary defini

 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.33 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       16.34 Angstroms and starts with atom:       1
   Please check group boundary definitions.
Minimize OMMD ligand copy 31
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.56 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.35 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.35 Angstroms and starts with atom:       1
   Please check group boundary definitions.
 NBONDA>>  Maximum group spatial extent (12A) exceeded.
   Size is       15.32 Angstroms and starts with atom:       1
   Please check group boundary defini

...

MINI>       20   -140.50019      0.70423      1.40663      0.00889
MINI INTERN>        4.23298     11.13568     13.01308     30.12802      0.00418
MINI EXTERN>      -52.42471   -146.58942      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       30   -140.79469      0.29451      0.65877      0.00517
MINI INTERN>        4.37393     11.43900     12.84512     29.98128      0.00527
MINI EXTERN>      -52.28682   -147.15247      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       40   -141.70960      0.91490      1.78996      0.01391
MINI INTERN>        4.56668     11.30167     13.06647     29.37299      0.00520
MINI EXTERN>      -50.72512   -149.29748      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
 UPDECI: Nonbond update at step        50
 Generating nonbond list with Exclusi

MINI>       10   -136.52206      1.29488      0.70045      0.00065
MINI INTERN>        3.70513     11.86172     13.76623     29.83639      0.01479
MINI EXTERN>      -48.62848   -147.07784      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       20   -136.84484      0.32278      1.03275      0.00070
MINI INTERN>        3.69891     11.73640     13.70114     29.81269      0.01455
MINI EXTERN>      -48.86709   -146.94144      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       30   -137.08663      0.24178      1.24794      0.00075
MINI INTERN>        3.67547     11.69056     13.65129     29.79253      0.01453
MINI EXTERN>      -49.05325   -146.85776      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>       40   -137.26631      0.17968      1.26380      0.00080
MINI INTERN>


 General atom nonbond list generation found:
    20659 ATOM PAIRS WERE FOUND FOR ATOM LIST
      420 GROUP PAIRS REQUIRED ATOM SEARCHES

MINI>      100   -145.61441      0.38121      0.57567      0.00538
MINI INTERN>        3.72511     11.60762     13.23988     32.04482      0.00932
MINI EXTERN>      -44.12864   -162.11252      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>      110   -145.90752      0.29310      0.74726      0.00521
MINI INTERN>        3.96919     11.45051     12.88537     32.07075      0.01076
MINI EXTERN>      -43.62640   -162.66769      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------
MINI>      120   -146.43561      0.52810      0.88661      0.00841
MINI INTERN>        3.79253     11.62129     13.03668     31.86186      0.01263
MINI EXTERN>      -43.40260   -163.35801      0.00000      0.00000      0.00000
 ----------       ---


 General atom nonbond list generation found:
    20878 ATOM PAIRS WERE FOUND FOR ATOM LIST
      417 GROUP PAIRS REQUIRED ATOM SEARCHES

MINI>       50   -139.16102      0.11531      0.36604      0.00036
MINI INTERN>        4.22204     13.46126     12.28721     26.80586      0.00504
MINI EXTERN>      -50.28557   -145.65686      0.00000      0.00000      0.00000
 ----------       ---------    ---------    ---------    ---------    ---------

 STEEPD> Minimization exiting with number of steps limit (        50) exceeded.

STPD MIN: Cycle      ENERgy      Delta-E         GRMS    Step-size
STPD INTERN:          BONDs       ANGLes       UREY-b    DIHEdrals    IMPRopers
STPD EXTERN:        VDWaals         ELEC       HBONds          ASP         USER
 ----------       ---------    ---------    ---------    ---------    ---------
STPD>       50   -139.16102      0.11531      0.36604      0.00043
STPD INTERN>        4.22204     13.46126     12.28721     26.80586      0.00504
STPD EXTERN>      -

Save top 10 lowest energy pose
   total_energy  grid_total     grid_vdw   grid_elec  grid_hbond  \
0   -131.107218 -115.815661   -49.096781  -66.718880         0.0   
1   -121.478764 -106.120143   -53.930132  -52.190011         0.0   
2   -120.425801 -105.707099   -56.212344  -49.494755         0.0   
3   -119.250130 -104.838296   -51.598947  -53.239349         0.0   
4   -118.142521 -104.605518   -53.006113  -51.599405         0.0   
5   -116.759579 -102.793137   -50.696240  -52.096897         0.0   
6   -112.093517 -100.179021   -44.029321  -56.149700         0.0   
7   -109.455953 -102.223175   -41.216468  -61.006707         0.0   
8   -102.754862  -97.086997   -43.266163  -53.820834         0.0   
9    671.566376  572.033974  1059.448479 -487.414505         0.0   

   conformer_id  placement_id  PDB_name cluster_size  
0            10             1  10_1.pdb            4  
1             5             5   5_5.pdb            5  
2             2             3   2_3.pdb            5  


### pyCHARMM Rigid CDOCKER Default Output

By default, pyCHARMM Rigid CDOCKER will create a folder (<font color='red'>dockresult</font>) and save all the outputs to this folder.

TSV files
1. <font color='red'>clusterResult.tsv</font> contains docking result for the clustered poses.
2. <font color='red'>explicitCluster.tsv</font> contains docking result for the clustered poses after explicit all atom minimization. 
3. <font color='red'>dockResult.tsv</font> contains docking result for all docked poses. 
4. <font color='red'>explicitTop10.tsv</font> contains docking result for the top 10 poses after explicit all atom minimization. 

Folders:
1. Folder <font color='red'>allPose/</font> contains all docked poses.
2. Folder <font color='red'>cluster/</font> contains all clustered poses.
3. Folder <font color='red'>top_ener/</font> contains the top 10 poses.


In [7]:
!ls dockresult

allPose  clusterResult.tsv  explicitCluster.tsv  top_ener
cluster  dockResult.tsv     explicitTop10.tsv


In [8]:
!ls dockresult/allPose dockresult/cluster dockresult/top_ener

dockresult/allPose:
10_1.pdb  1_3.pdb  2_5.pdb  4_2.pdb  5_4.pdb  7_1.pdb  8_3.pdb	9_5.pdb
10_2.pdb  1_4.pdb  3_1.pdb  4_3.pdb  5_5.pdb  7_2.pdb  8_4.pdb
10_3.pdb  1_5.pdb  3_2.pdb  4_4.pdb  6_1.pdb  7_3.pdb  8_5.pdb
10_4.pdb  2_1.pdb  3_3.pdb  4_5.pdb  6_2.pdb  7_4.pdb  9_1.pdb
10_5.pdb  2_2.pdb  3_4.pdb  5_1.pdb  6_3.pdb  7_5.pdb  9_2.pdb
1_1.pdb   2_3.pdb  3_5.pdb  5_2.pdb  6_4.pdb  8_1.pdb  9_3.pdb
1_2.pdb   2_4.pdb  4_1.pdb  5_3.pdb  6_5.pdb  8_2.pdb  9_4.pdb

dockresult/cluster:
top_10.pdb  top_2.pdb  top_4.pdb  top_6.pdb  top_8.pdb
top_1.pdb   top_3.pdb  top_5.pdb  top_7.pdb  top_9.pdb

dockresult/top_ener:
top_10.pdb  top_2.pdb  top_4.pdb  top_6.pdb  top_8.pdb
top_1.pdb   top_3.pdb  top_5.pdb  top_7.pdb  top_9.pdb


1. In the folder <font color='red'>allPose/</font>, the nomenclature of the pdb files is \{conformer_id\}_\{placement_id\}
2. In the folder <font color='red'>cluster/</font> and <font color='red'>top_ener</font>, the docked poses are sorted (i.e., top_1.pdb to top_n.pdb). The sorting method will be discussed later.

#### Lets look at the docked pose first

In [70]:
import py3Dmol
view = py3Dmol.view(width=500, height=400)
view.setBackgroundColor('grey')
for i,struct in enumerate(['ligand.pdb','protein.pdb','dockresult/cluster/top_1.pdb',
                          'dockresult/top_ener/top_1.pdb']):
    view.addModel(open(struct,'r').read(),'pdb')
    if i > 1: 
        view.setStyle({'model':i},{'stick':{'colorscheme':'greenCarbon'}})
    elif i == 0:
        view.setStyle({'model':i},{'stick':{'colorscheme':'yellowCarbon' }})
        view.addSurface(py3Dmol.VDW,{'opacity': 0.7},{'model':i})
    elif i == 1:
        view.setStyle({'model':i},{'cartoon':{'color': 'lightblue'}})
        view.addSurface(py3Dmol.SES,{'opacity': 0.8,'color': 'white'},{'model':i})
view.zoomTo()
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Now lets look at these tsv files. 

In [ ]:
import pandas as pd

1. dockResult.tsv will always be created. It contains the docking results for all docked poses and is sorted by <font color='red'>total energy</font>. 

In [ ]:
print(pd.read_csv("dockresult/dockResult.tsv", sep ='\t').drop("Unnamed: 0", axis = 1))

2. clusterResult.tsv will always be created. It contains the docking results for the clustered poses and is sorted by <font color='red'>total energy</font>

In [ ]:
print(pd.read_csv("dockresult/clusterResult.tsv", sep = '\t').drop("Unnamed: 0", axis = 1))

3. By default, cluster representatives and top 10 poses will be rescored with explicit receptor atoms.
The user can decide how they want to sort these two dataframes with the parameter <font color='red'>sort_energy</font>.
The default sorting is based on total energy. 


In [ ]:
explicitTop10 = pd.read_csv("dockresult/explicitTop10.tsv", sep = '\t').drop("Unnamed: 0", axis = 1)
explicitCluster = pd.read_csv("dockresult/explicitCluster.tsv", sep = '\t').drop("Unnamed: 0", axis = 1)
print("Top 10 lowest energy poses with explicit all atom minimization. \n \n", explicitTop10, "\n \n")
print("Cluster poses with explicit all atom minimization. \n \n", explicitCluster)

## pyCHARMM Rigid CDOCKER Parameter for Controlling Input Information. 

1. Parameters that require user input

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| xcen | 0 | x coordinate of the center of the grid box | 
| ycen | 0 | y coordinate of the center of the grid box | 
| zcen | 0 | z coordinate of the center of the grid box | 
| maxlen | 10 | size of the grid box | 

2. Parameters that are miscellaneous. 

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| ligPDB | 'ligand.pdb' | ligand PDB file | 
| ligSeg | 'LIGA' | Segment ID in the ligand PDB file | 
| receptorPDB | 'protein.pdb' | receptor PDB file | 
| receptorPSF | 'protein.psf' | receptor PSF file | 
| numPlace | 100 | number of placement per conformer | 
| sort_energy | 'total_energy' | sorting method of the explicit all atom minimization result |
| saveDir | './dockresult/' | folder name of the final result directory |
| confDir | './conformer/' | folder name of the conformer library |
| flag_center_ligand | Ture | whether or not center the ligand in the binding pocket | 
| flag_grid | False | whether or not grid need to be generated before docking |
| flag_delete_grid | True | whether or not delete grid after docking |
| flag_delete_conformer | True | whether or not delete conformer after docking |
| flag_delete_placement | True | whether or not delete ligand initial placement after docking| 
| flag_save_all | Ture | whether or not save all docked pose after docking |
| flag_save_cluster | True | whether or not save clustered results after docking |
| flag_save_top | Ture | whether or not save top 10 ranking poses after docking |

3. Parameters that are not recommended to be changed and the default values are applicable for most use cases.

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| dielec | 3.0 | dielectric constant | 
| flag_rdie | True | True for rdie, False for cdie | 
| flag_form | False | whether or not grid form is formatted | 
| probeFile | "../Toppar/fftdock_c36prot_cgenff_probes.txt" | probe file for gpu grid generation| 
| softGridFile | 'grid-emax-0.6-mine--0.4-maxe-0.4.bin' | soft grid file name |
| hardGridFile | 'grid-emax-3-mine--30-maxe-30.bin' | hard grid file name |
| nativeGridFile | 'grid-emax-100-mine--100-maxe-100.bin' | native grid file name |
| placementDir | './placement/' | ligand placement folder name |
| flag_fast_placement | True | True for using fast placement, False for using original Rigid CDOCKER placement method |
| exhaustiveness | 'high' | exhaustiveness for fast placement, high, medium, low |
| threshold | 2500 | cutoff threshold for original Rigid CDOCKER placement method |
| numCopy | 1000 | maximum number of copy for OpenMM parallel simulated annealing |
| flag_suppress_print | True | whether or not suppress warning message during docking |
| flag_fast_grid | False | True for using grid minimization, False for explicit atom minimization |

# End of rigid CDOCKER example - return home

In [ ]:
os.chdir(home)
os.getcwd()

# pyCHARMM Rigid CDOCKER Covalent Docking

The major difference between the covalent docking and non-covalent docking in the pyCHARMM Rigid CDOCKER is that the user needs to <font color=red>specify the well-depth and cutoffs</font> of the covalent bond grid potential. Below shows the customizable covalent (hydrogen bond) grid potential.

***Note***
1. User needs to specifiy the sets of ligand and receptor atoms that are interested. 
2. The corresponding parameter files need to be updated
3. CHARMM selection script: 
       1. acceptor set select *** end
       2. donor set select *** end

<img src="figure/ener.png" width="80%"/>

And the corrsponding parameters in the Rigid_CDOCKER are :

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| rcta | 0 | customizable grid left cutoff | 
| rctb | 0 | customizable grid right cutoff |
| hmax | 0 | customizable grid well-depth |
| flag_use_hbond | False | whether or not use hydrogen/covalent bond grid potential in Rigid CDOCKER | 

# pyCHARMM Flexible CDOCKER

## Helper function

In [ ]:
from pycharmm.cdocker import Flexible_CDOCKER
help(Flexible_CDOCKER)

## pyCHARMM Flexible CDOCKER Docking and Analysis

### Running Flexible CDOCKER with Default Input

1. ligand file: <font color='red'>ligand.pdb</font>
2. ligand segment ID: <font color='red'>LIGA</font>
3. receptor file : <font color='red'>protein.pdb</font> and <font color='red'>protein.psf</font>
4. Default docking trials: <font color='red'>20</font> conformers generated by Open Babel and <font color='red'>25</font> copies each conformer. (i.e., Open Babel is essential for Flexible CDOCKER)

With default input, the user only needs to provide <font color='red'>grid box information</font> and <font color='red'>flexible amino acid side chain selection</font> (see example below).

***Note***
1. Currently, pyCHARMM Flexible CDOCKER only works with protein receptor.
2. The flexible side chain selection needs to be a pandas dataframe. 

In [ ]:
import pandas as pd
flexchain = pd.read_csv('../flex/flexchain.csv', sep = '\t', index_col = 0)
print(flexchain)

With default input, the user only need to provide <font color='red'>grid box information</font> and <font color='red'>flexible side chain selection</font> (see example below).

```Flexible_CDOCKER(xcen = 1, ycen = 1, zcen = 1, maxlen = 10, flexchain = flexchain)```

### pyCHARMM Flexible CDOCKER Example

We have shown some basic requirments for Flexible CDOCKER. Below is a practical example of perform pyCHARMM Flexible CDOCKER docking experiments.
    
***Note***
1. In this example, we specified the <font color='red'>file location</font> of the ligand, receptor and flexible side chain selection (i.e., they are not at their default location).
2. We also change the number of conformers to <font color='red'>5</font> and the number of docking trials per conformer to <font color='red'>5</font> to reduce the computational cost. 

In [1]:
# reset the kernel to run another system
%reset -f
import os
os.chdir('../jupyter_lab')
home = os.getcwd()
if not os.path.exists('../test_flex'): os.mkdir('../test_flex')
os.chdir('../test_flex')
flex = os.getcwd()
print(home,flex)
# Move ligand.pdb, ligandrtf, protein.pdb and protein.psf to working directory
os.system(f'cp ../flex/ligand.* {flex}/')
os.system(f'cp ../flex/protein* {flex}/')
os.system(f'cp ../flex/flexchain.csv {flex}/')
# Finally, move the pre-compiuted grid files to the working directory
os.system(f'cp {home}/grid/flex/* {flex}/')

/home/brookscl/pyCHARMM-Workshop/CDOCKER_WYu/jupyter_lab /home/brookscl/pyCHARMM-Workshop/CDOCKER_WYu/test_flex


0

In [2]:
os.environ['CHARMM_LIB_DIR'] = '/home/brookscl/charmm/c47-dev-release/install-pycharmm-c1201-52/lib/'
## Import module
import pandas as pd
import pycharmm
import pycharmm.lib as lib
import pycharmm.read as read
import pycharmm.lingo as lingo
import pycharmm.settings as settings
from pycharmm.cdocker import Flexible_CDOCKER

## File name and pathway
ligPDB = "ligand.pdb"
ligandrtf = "ligandrtf"
receptorPDB = "protein.pdb"
receptorPSF = "protein.psf"
topdir = "../Toppar/"

settings.set_bomb_level(-1)
owl = settings.set_warn_level(-5)
read.rtf('"../Toppar/top_all36_prot.rtf"')
read.rtf('"../Toppar/top_all36_cgenff.rtf"', append = True)
read.prm('"../Toppar/par_all36m_prot.prm"', flex = True)
read.prm('"../Toppar/par_all36_cgenff.prm"', append = True, flex = True)
settings.set_bomb_level(0)
read.stream('ligandrtf')
settings.set_warn_level(owl)

## Read in the receptor flexible side chain selection
flexchain = pd.read_csv('flexchain.csv', sep = '\t', index_col = 0)

## Flexible CDOCKER standard docking protocol
clusterResult, dockResult = Flexible_CDOCKER(xcen = 26.911, ycen = 6.126, zcen = 4.178,
                            maxlen = 14.492, ligPDB = ligPDB, receptorPDB = receptorPDB,
                            receptorPSF = receptorPSF, num = 5, copy = 5,
                            flag_grid = True, flexchain = flexchain)

print(clusterResult)
print(dockResult)

  
 CHARMM>     read rtf card -
 CHARMM>     name "../Toppar/top_all36_prot.rtf"
 VOPEN> Attempting to open::../Toppar/top_all36_prot.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *>>>>>>>>CHARMM36 ALL-HYDROGEN TOPOLOGY FILE FOR PROTEINS <<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> MAY 2011 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read rtf card -
 CHARMM>     name "../Toppar/top_all36_cgenff.rtf" -
 CHARMM>     append
 VOPEN> Attempting to open::../Toppar/top_all36_cgenff.rtf::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *  --------------------------------------------------------------------------  *
 TITLE> *          CGENFF: TOPOLOGY FOR THE CHARMM GENERAL FORCE FIELD V. 4.3  


 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:        157 residues deleted.
 DELTIC:      2548 bonds deleted
 DELTIC:      4604 angles deleted
 DELTIC:      6727 dihedrals deleted
 DELTIC:       450 improper dihedrals deleted
 DELTIC:       160 crossterm maps deleted
 DELTIC:       320 donors deleted
 DELTIC:       235 acceptors deleted
  
 CHARMM>     read sequence pdb -
 CHARMM>     name ligand.pdb
 VOPEN> Attempting to open::ligand.pdb::
 MAINIO> Sequence information being read from unit  91.
 TITLE>   TITLE
 TITLE>   MOVE LIGAND TO THE CENTER OF BINDING POCKET
 TITLE>    DATE:     6/24/23     15: 3:36      CREATED BY USER: BROOKSCL
 TITLE>  *

          RESIDUE SEQUENCE --     1 RESIDUES
          I4B     
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 NO PATCHING WILL BE DONE ON THE FIRST RESIDUE
 NO PATCHING WILL BE DONE ON THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   2 has been generated. 

*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

A selection has been stored as MZOHCBFRMAASCXNJODIA
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  1
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  2
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  3
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  4
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  5


*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 1)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 3)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 4)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PD

...

A selection has been stored as ZVFLPYRDNPXOHRVXZVXX
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  21
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  22
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  23
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  24
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  25
Set OMMD ligand copy 1
Set OMMD ligand copy 2
Set OMMD ligand copy 3
Set OMMD ligand copy 4
Set OMMD ligand copy 5
Set OMMD ligand copy 6
Set OMMD ligand copy 7
Set OMMD ligand copy 8
Set OMMD ligand copy 9
Set OMMD ligand copy 10
Set OMMD ligand copy 11
Set OMMD ligand copy 12
Set OMMD ligand copy 13
Set OMMD ligand copy 14
Set OMMD ligand copy 15
Set OMMD ligand copy 16
Set OMMD ligand co

/export/apps/CentOS7/mmtsb/src/feig/toolset//bin/kclust -pdb -centroid -cdist -radius 1 -iterate -maxerr 0.01 -mode rmsd -heavy


Flexible docking for generation: 2
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  1
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  2
Flexible receptor + ligand placement  3
Flexible receptor + ligand placement  4
Flexible receptor + ligand placement  5
Flexible receptor + ligand placement  6
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  7
Flexible receptor + ligand placement  8
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  9
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + ligand placement  10
Flexible receptor + ligand placement  11
Flexible receptor + ligand placement  12
Flexible receptor + ligand placement  13
Grid is applied to specified atoms
Grid is applied to specified atoms
Flexible receptor + l

/export/apps/CentOS7/mmtsb/src/feig/toolset//bin/kclust -pdb -centroid -cdist -radius 1 -iterate -maxerr 0.01 -mode rmsd -heavy


Calculate conformational entropy contribution from amino acid  LEU
Calculate conformational entropy contribution from amino acid  VAL
Calculate conformational entropy contribution from amino acid  VAL
Calculate conformational entropy contribution from amino acid  LEU

 Message from MAPIC: Atom numbers are changed.

 Message from MAPIC:          6 residues deleted.

 Message from MAPIC:          2 segments deleted.
 DELTIC:        99 bonds deleted
 DELTIC:       172 angles deleted
 DELTIC:       222 dihedrals deleted
 DELTIC:         5 donors deleted
 DELTIC:         5 acceptors deleted
  
 CHARMM>     read psf card -
 CHARMM>     name protein.psf -
 CHARMM>     append
 VOPEN> Attempting to open::protein.psf::
 MAINIO> Protein structure file being appended from unit  91.
 TITLE>  * BUILD PROTEIN PSF AND PDB FILE
 TITLE>  *  DATE:    12/ 3/21     13:16: 4      CREATED BY USER: yujin
 TITLE>  *
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure

### pyCHARMM Flexible CDOCKER Default Output

By default, pyCHARMM Rigid CDOCKER will create a folder (<font color='red'>dockresult</font>) and save all the outputs to this folder.

TSV files
1. <font color='red'>clusterResult.tsv</font> contains docking result for the clustered poses.
2. <font color='red'>explicitResult.tsv</font> contains docking result for all docked poses after explicit all atom minimization. 
3. <font color='red'>dockResult.tsv</font> contains docking result for all docked poses. 

Folders:
1. Folder <font color='red'>cluster/</font> contains all clustered poses.

In [3]:
!ls dockresult

cluster  cluster.log  clusterResult.tsv  dockResult.tsv  explicitResult.tsv


In [4]:
!ls dockresult/cluster/*

dockresult/cluster/ligand:
top_1.pdb  top_2.pdb  top_3.pdb  top_4.pdb

dockresult/cluster/protein:
top_1.pdb  top_2.pdb  top_3.pdb  top_4.pdb


In the folder <font color='red'>cluster/</font>, the ligand and the corresponding protein docked poses are sorted (i.e., top_1.pdb to top_n.pdb). The sorting method will be introduced later.

#### Lets look at the docked pose first

In [7]:
import py3Dmol
view = py3Dmol.view(width=500, height=400)
view.setBackgroundColor('grey')
for i,struct in enumerate(['ligand.pdb','dockresult/cluster/ligand/top_1.pdb',
                           'protein.pdb']):
    view.addModel(open(struct,'r').read(),'pdb')
    if i == 1: 
        view.setStyle({'model':i},{'stick':{'colorscheme':'greenCarbon'}})
    elif i == 0:
        view.setStyle({'model':i},{'stick':{'colorscheme':'yellowCarbon' }})
        view.addSurface(py3Dmol.VDW,{'opacity': 0.7},{'model':i})
    elif i == 2:
        view.setStyle({'model':i},{'cartoon':{'color': 'lightblue'}})
        #view.addSurface(py3Dmol.SES,{'opacity': 0.8,'color': 'white'},{'model':i})
view.zoomTo()
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

#### Now lets look at these tsv files. 

In [ ]:
import pandas as pd

1. dockResult.tsv will always be created. It contains the docking results for all docked poses and is sorted by <font color='red'>total energy</font>. 

In [ ]:
print(pd.read_csv("dockresult/dockResult.tsv", sep ='\t').drop("Unnamed: 0", axis = 1))

2. clusterResult.tsv will always be created. It contains the docking representatives for the clustered poses and is sorted by <font color='red'>total energy</font>. By default, these poses will be rescored with explicit receptor atoms.The user can decide how they want to sort the dataframe with the parameter <font color='red'>sort_energy</font>. The default sorting is based on total energy. 

In [ ]:
print(pd.read_csv("dockresult/clusterResult.tsv", sep = '\t').drop("Unnamed: 0", axis = 1))

3. By default, all docked poses will be rescored with explicit receptor atoms. The result is sorted by <font color='red'>total energy</font> and saved in the file explicitResult.tsv. The user can decide how they want to sort the dataframe with the parameter <font color='red'>sort_energy</font>. The default sorting is based on total energy. 

In [ ]:
print(pd.read_csv("dockresult/explicitResult.tsv", sep = '\t').drop("Unnamed: 0", axis = 1))

## pyCHARMM Flexible CDOCKER Parameter for Controlling Input Information. 

1. Parameters that require user input

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| xcen | 0 | x coordinate of the center of the grid box | 
| ycen | 0 | y coordinate of the center of the grid box | 
| zcen | 0 | z coordinate of the center of the grid box | 
| maxlen | 10 | size of the grid box | 
| flexchain | None | flexible side chain selection (pandas dataframe)|

2. Parameters that are miscellaneous. 

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| ligPDB | 'ligand.pdb' | ligand PDB file | 
| ligSeg | 'LIGA' | Segment ID in the ligand PDB file | 
| receptorPDB | 'protein.pdb' | receptor PDB file | 
| receptorPSF | 'protein.psf' | receptor PSF file | 
| num | 20 | number of conformers | 
| copy | 25 | number of copies per conformer |
| generation | 2 | number of generations in the genetic algorithm |
| sort_energy | 'total_energy' | sorting method of the explicit all atom minimization result |
| saveDir | './dockresult/' | folder name of the final result directory |
| flag_center_ligand | Ture | whether or not center the ligand in the binding pocket | 
| flag_grid | False | whether or not grid need to be generated before docking |
| flag_delete_grid | True | whether or not delete grid after docking |
| flag_save_placement | True | whether or not save ligand initial placement after docking| 
| flag_save_crossover | False | whether or not save crossover after docking| 
| flag_save_all | False | whether or not save all docked pose after docking |
| flag_save_cluster | True | whether or not save clustered results after docking |
| flag_save_top | Ture | whether or not save top 10 ranking poses after docking |
| top_N_result | 10 | number of top N largest clusters, final generation uses top N + 5 clusters | 

3. Parameters that are not recommended to be changed and the default values are applicable for most use cases.

| Parameters | Default value | Meaning|
| :-- | :-- | :-- |
| flag_form | False | whether or not grid form is formatted | 
| probeFile | "../Toppar/fftdock_c36prot_cgenff_probes.txt" | probe file for gpu grid generation| 
| softGridFile | 'grid-emax-0.6-mine--0.4-maxe-0.4.bin' | soft grid file name |
| hardGridFile | 'grid-emax-3-mine--30-maxe-30.bin' | hard grid file name |
| nativeGridFile | 'grid-emax-100-mine--100-maxe-100.bin' | native grid file name |
| receptorCard | '"./flexchain.crd"' | receptor flexible side chain coordinate card |
| saveLig | './ligand/' | folder name for all ligand docked poses |
| saveProt | './protein/' | folder name for all protein docked poses |
| saveLigFinal | './ligand_final/' | folder name for ligand final cluster representatives |
| saveProtFinal | './protein_final/' | folder name for protein final cluster representatives |
| crossoverLig | './crossover_ligand/' | folder name for ligand crossover |
| crossoverProt | './crossover_protein/' | folder name for protein crossover |
| placementDir | './placement/' | ligand placement folder name |
| threshold_init | 2500 | energy threshold for initial placement |
| threshold_mutate | 100 | energy threshold for mutation |
| flag_fast_grid | False | True for using grid minimization, False for explicit atom minimization |
| flag_fast_placement | False | True for using fast placement, False for using original Flexible CDOCKER placement method |
| exhaustiveness | 'high' | exhaustiveness for fast placement, high, medium, low |
| flag_suppress_print | True | whether or not suppress warning message during docking | 

# PyCHARMM CDOCKER Docking Summary

## Docking Speed

1. Slightly improve Rigid CDOCKER docking speed.
2. Significantly improved Flexible CDOCKER docking speed by at least 2 folds. 

<img src='figure/fig_pycharmm_time.png' width=50%></img>

## Docking Accuracy

Because we improved the searching efficiency, we could perform more exhaustive searching. And hence we observe improved pose prediction performance. 

    1. We compared the Rigid CDOCKER docking performance with the benchmark dataset SB2012.
    2. We compared the Flexible CDOCKER docking preformance with the cross-docking benchmark dataset. 

<img src='figure/fig_pycdocker.png' width=80%></img>

## Summary

    1. Reduce the complexity
    2. Accelerate CDOCKER family
    3. Improve docking accuracy
    4. Easy to integrate with other python packages

## Future work

    1. pyCHARMM Flexible CDOCKER atom selection
    2. FFT docking in pyCHARMM
    3. One liner code for virtual screening